In [97]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import datetime
from sklearn.pipeline import FeatureUnion

# Split training and testing dataset

In [98]:
data=pd.read_pickle('data.pkl')

In [99]:
train=data[data.index<=datetime.datetime(2015, 5, 4)]
test=data[data.index>datetime.datetime(2015, 5, 4)]

In [100]:
train.head()

,IC,NOS,PCR,BDI,BM,SPX,SI,CAY,BY,DP,...,CAPE,PCAPrice,DEF,VRP,OIL,CPI,R_1M,R_3M,R_6M,R_12M
AsOfDate,,,,,,,,,,,,,,,,,,,,,
1990-06-08,NaN,0.016941,-1.622923,1278.0,NaN,-0.012226,NaN,-0.013576,1.007418,0.063979,...,8.978524,3.616905,0.96,7.197727,NaN,0.046737,0.002258,-0.117555,-0.093845,0.044830
1990-06-11,NaN,0.016941,-1.615366,1264.0,NaN,0.008140,NaN,-0.013576,1.009642,0.063463,...,9.051612,3.583706,0.96,5.607431,NaN,0.046737,-0.006189,-0.098436,-0.086309,0.055365
1990-06-12,NaN,0.016941,-1.618868,1254.0,NaN,0.012775,NaN,-0.013576,1.009580,0.062662,...,9.167251,3.532720,0.96,6.403252,NaN,0.046737,-0.001106,-0.110610,-0.090534,0.053701
1990-06-13,NaN,0.016941,-1.625316,1243.0,NaN,-0.003686,NaN,-0.013576,1.000056,0.062894,...,9.133460,3.547245,0.96,7.793147,NaN,0.046737,-0.002212,-0.123440,-0.108696,0.028396
1990-06-14,NaN,0.016941,-1.620454,1232.0,NaN,-0.005481,NaN,-0.013576,0.997689,0.063241,...,9.083400,3.569002,0.96,7.665179,NaN,0.046737,0.006605,-0.116087,-0.095122,0.034886


# Prepocess

In [101]:
#pipelines to select columns
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]
#de-median
class DeMedian(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        self.median=0
    def fit(self, X):
        self.median=X.median()
        return self  # nothing else to do
    def transform(self, X):
        return X-self.median
#take log
class Log(BaseEstimator, TransformerMixin):
    def __init__(self): 
        return
    def fit(self, X):
        return self
    def transform(self, X):
        return np.log(X)

In [110]:
feature_list=['DP','PE','BM','CAPE','PCAPrice','BY','DEF','TERM','CAY',
              #'SIM',
              'VRP','IC',
              'BDI','NOS','CPI','PCR',
              #'MA','PCA-tech',
              'OIL','SI'] 
R_list=['R_1M','R_3M','R_6M','R_12M']

demedian_list=['VRP','IC']
log_median_list=['BDI']

other_list=[i for i in feature_list if i not in demedian_list+log_median_list]
demedian_pipeline = Pipeline([
        ('selector', DataFrameSelector(demedian_list)),
        ('demedian', DeMedian()),
    ])

log_median_pipeline = Pipeline([
        ('selector', DataFrameSelector(log_median_list)),
        ('log', Log()),
        ('demedian', DeMedian()),
    ])
other_pipeline=Pipeline([
    ('selector', DataFrameSelector(other_list)),
])
full_pipeline = FeatureUnion(transformer_list=[
        ("demedian_pipline", demedian_pipeline),
        ("log_median_pipeline", log_median_pipeline),
        ("other_pipeline", other_pipeline),
        
    ])

In [111]:
train_prepared=full_pipeline.fit_transform(train)

In [112]:
train_prepared

array([[ 0.73708797,         nan, -0.1528084 , ..., -1.62292303,
                nan,         nan],
       [-0.85320834,         nan, -0.16382346, ..., -1.6153656 ,
                nan,         nan],
       [-0.0573871 ,         nan, -0.17176631, ..., -1.61886771,
                nan,         nan],
       ...,
       [-0.64222115, -4.3       , -0.92404402, ..., -0.4574592 ,
        -0.52399892,  0.98136784],
       [-2.57190703, -6.98      , -0.93083521, ..., -0.44264525,
        -0.52399892,  0.53942159],
       [-2.33401527, -5.21      , -0.93083521, ..., -0.44017394,
        -0.52399892,  0.48065865]])

# Tables

In [113]:
df_prepared=pd.DataFrame(train_prepared,columns=demedian_list+log_median_list+other_list,index=train.index)

## Table1

In [114]:
df_prepared[R_list]=train[R_list]

In [115]:
summary=df_prepared.describe().T
summary.loc[:,'Skewness']=df_prepared.skew()
summary.loc[:,'Kurtosis']=df_prepared.kurt()
summary.loc[feature_list+R_list]

,count,mean,std,min,25%,50%,75%,max,Skewness,Kurtosis
DP,6275.0,0.030313,0.013598,0.016091,0.021619,0.023294,0.036607,0.077506,1.376078,0.807164
PE,6275.0,17.149349,8.268075,6.632500,13.329156,15.274371,18.615715,75.827909,3.939150,20.689034
BM,3858.0,0.382313,0.081155,0.190303,0.342486,0.367829,0.446324,0.683908,0.091054,0.219331
CAPE,6275.0,18.176844,5.022590,7.395374,14.564245,18.992797,21.157242,30.218934,-0.041247,-0.509246
PCAPrice,6275.0,0.225213,1.545274,-3.531514,-0.769562,-0.478372,1.497023,4.402838,0.823031,-0.244657
BY,6275.0,0.973717,0.100592,0.552394,0.914494,0.973263,1.036844,1.393599,-0.063666,1.270132
DEF,6275.0,0.954183,0.410862,0.550000,0.690000,0.870000,1.060000,3.380000,3.140201,12.986900
TERM,6272.0,1.865515,1.161928,-0.950000,0.910000,2.000000,2.790000,3.870000,-0.290813,-0.984421
CAY,6275.0,0.005230,0.025816,-0.052577,-0.009389,0.004489,0.018684,0.091062,0.601011,1.697840
VRP,6275.0,0.430350,3.458404,-24.511136,-1.740792,0.000000,2.131014,23.839293,0.703677,4.079881


## Table2

In [117]:
df_prepared[feature_list].corr()

,DP,PE,BM,CAPE,PCAPrice,BY,DEF,TERM,CAY,VRP,IC,BDI,NOS,CPI,PCR,OIL,SI
DP,1.000000,-0.399938,0.713363,-0.881866,0.937773,-0.031397,0.049215,0.240396,-0.036017,0.064234,0.269369,-0.151734,-0.098631,0.371099,-0.623930,-0.289627,0.142280
PE,-0.399938,1.000000,0.087505,0.260784,-0.629854,0.138330,0.110652,0.281411,0.440635,0.119581,0.020482,0.151162,-0.319521,-0.383968,0.280559,-0.021459,-0.133759
BM,0.713363,0.087505,1.000000,-0.771820,0.217037,-0.283958,0.600965,0.448422,0.752290,0.309994,0.705814,-0.074796,-0.136490,-0.365407,-0.336642,-0.320335,0.059417
CAPE,-0.881866,0.260784,-0.771820,1.000000,-0.792653,0.122016,-0.210838,-0.449741,-0.333042,-0.074996,-0.485870,0.008292,0.138378,-0.230004,0.776953,0.173875,-0.016728
PCAPrice,0.937773,-0.629854,0.217037,-0.792653,1.000000,-0.043157,-0.117171,0.076835,-0.241570,0.003178,0.084688,-0.202552,-0.022177,0.418389,-0.564653,-0.264859,0.060071
BY,-0.031397,0.138330,-0.283958,0.122016,-0.043157,1.000000,-0.329162,0.092459,-0.018819,-0.181390,-0.478965,0.106304,0.090832,0.050213,0.028541,0.099025,-0.072033
DEF,0.049215,0.110652,0.600965,-0.210838,-0.117171,-0.329162,1.000000,0.253921,0.500169,0.246082,0.330308,0.055089,-0.249626,-0.243199,-0.172638,0.047364,-0.029534
TERM,0.240396,0.281411,0.448422,-0.449741,0.076835,0.092459,0.253921,1.000000,0.542635,0.092356,0.398803,-0.071969,-0.238081,-0.238530,-0.238590,-0.231822,-0.053362
CAY,-0.036017,0.440635,0.752290,-0.333042,-0.241570,-0.018819,0.500169,0.542635,1.000000,0.075844,0.341942,0.302608,-0.161129,-0.324996,-0.313343,0.013124,-0.069644
VRP,0.064234,0.119581,0.309994,-0.074996,0.003178,-0.181390,0.246082,0.092356,0.075844,1.000000,0.351058,-0.108147,-0.209388,-0.098929,0.031323,-0.174315,0.454565


### Table 3

In [118]:
df_prepared.corr().loc[feature_list,R_list]

,R_1M,R_3M,R_6M,R_12M
DP,0.056295,0.126790,0.221905,0.288785
PE,-0.013689,-0.042307,-0.104193,-0.083680
BM,0.154924,0.275348,0.431912,0.587916
CAPE,-0.075022,-0.149799,-0.256317,-0.376621
PCAPrice,0.063974,0.137270,0.234523,0.303291
BY,-0.039399,-0.038604,-0.035706,0.032861
DEF,-0.062446,-0.068539,-0.008185,0.049405
TERM,-0.032893,-0.057962,-0.044279,0.056567
CAY,0.038806,0.062604,0.083401,0.171004
VRP,0.035292,0.153809,0.153953,0.144068
